In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Use sqlite3

**Prep Up Environment**

In [ ]:
import sqlite3

In [ ]:
# connect to the db file, the provided db file has already contains all the tables needed for the quiz
conn = sqlite3.connect('sqlite.db')

In [ ]:
cur = conn.cursor()

In [ ]:
# show what tables in the database. 
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchall()

[('clicks',),
 ('sqlite_autoindex_clicks_1',),
 ('documents_meta',),
 ('events',),
 ('promoted_content',)]

***Notes***

platform (desktop = 1, mobile = 2, tablet =3)

clicked (1 if clicked, 0 otherwise)

## Question-1##

Retrieve all display events shown in tablet (platform). Please use events table and return all attributes of 
the display event that satisfy the condition.


In your solution, please provide the SQL query which can best answer the question. Also, please answer 
how many rows are in the returned results of the provided query. 

In [ ]:
data=cur.execute("select a.* from events a where a.platform=3 limit 5")
data.fetchall()

[(1, 'cb8c55702adb93', 379743, 61, 3, 'US>SC>519'),
 (6, '7765b4faae4ad4', 1773517, 395, 3, 'US>OH>510'),
 (11, '602e210c5831e5', 1773230, 710, 3, 'US>IL>675'),
 (14, 'daef797fc210a2', 1759953, 798, 3, 'US>NC>560'),
 (19, 'c419799a427c72', 1155107, 1202, 3, 'US>HI>744')]

In [ ]:
data=cur.execute("select count(*) from (select a.* from events a where a.platform=3)")
data.fetchall()

[(320,)]

##Question-2##

Retrieve all display events that is clicked and shown in desktop (platform). Please return all attributes of 
the display event that satisfy the condition. (15 points)

In your solution, please provide the SQL query which can best answer the question. Also, please answer 
how many rows are in the returned results of the provided query. 

In [ ]:
data=cur.execute("select a.* from events a inner join clicks b on a.display_id=b.display_id where a.platform=1 and b.clicked=1 limit 5")
data.fetchall()

[(9, '9dddccf70f6067', 1772126, 667, 1, 'US>FL>528'),
 (10, 'b09a0e92aa4d17', 157455, 693, 1, 'US'),
 (12, '6fa993bd0e0157', 892701, 718, 1, 'US>TX>612'),
 (13, '7355615832b3af', 1778533, 739, 1, 'US>AZ>753'),
 (16, '30c0ad12b36375', 1727882, 1033, 1, 'US>FL>561')]

In [ ]:
data=cur.execute("select count(*) from (select a.* from events a inner join clicks b on a.display_id=b.display_id where a.platform=1 and b.clicked=1)")
data.fetchall()

[(617,)]

##Question-3.1##

3.1 Please use one SQL query to find out the following answers: 

per platform how many unique display 
events have been clicked? 

per platform how many unique display events have not been clicked? 

Unique events can be identified according to display_id. 

For query result, please return whether the events is 
clicked or not, platform, the counts asked in the questions. 

If the count is 0, no need to include the tuple in 
the query result. (10 points)


**Interpretation**
The ask is to find how many unique events(display_id) have been clicked or not  per platform.

The results are
Clicked, Platform, #Unique count(display_id)

In [ ]:
data=cur.execute("select b.clicked,a.platform,count(distinct b.display_id) from events a inner join clicks b on a.display_id=b.display_id where b.clicked='1' group by b.clicked,a.platform union select b.clicked,a.platform,count(distinct b.display_id) from events a inner join clicks b on a.display_id=b.display_id where b.clicked='0' and b.display_id not in (select display_id from Clicks where  clicked='1' ) group by b.clicked, a.platform  ")
data.fetchall()

[(1, 1, 617), (1, 2, 1022), (1, 3, 320)]

##Question 3.2##
Please use one SQL query to find out the following answers: 

per platform how many unique ads 
events have been clicked? 

per platform how many unique ads events have not been clicked? 

Unique 
events can be identified according to ad_id.

 For query result, please return whether the events is clicked or 
not, platform, the counts asked in the questions. If the count is 0, no need to include the tuple in the query 
result. (5 points)


In [ ]:
data=cur.execute("select b.clicked,a.platform,count(distinct b.ad_id) from events a inner join clicks b on a.display_id=b.display_id where b.clicked='1'  and b.ad_id not in (select ad_id from Clicks a where clicked='0') group by b.clicked,a.platform union select b.clicked,a.platform,count(distinct b.display_id) from events a inner join clicks b on a.display_id=b.display_id where b.clicked='0' and b.ad_id not in (select ad_id from Clicks where  clicked='1' ) group by b.clicked, a.platform  ")
data.fetchall()

[(0, 1, 599), (0, 2, 900), (0, 3, 308), (1, 1, 178), (1, 2, 260), (1, 3, 86)]

##Question 4##

Exam all advertiser's ads performance in different platforms. We use click through rate to indicate ads 
performance.

 Click through rate is defined as the number of ad events have been clicked divided by the 
number of all ad events. 

SQL query needs to return ad_id, platform, click through rate. 
In the SQL query 
result, the ad and platform with highest click through rate needs to be shown first, and then show the ad 
and platform with the second highest click through rate, so on so forth, until the the ad and platform with 
lowest click through rate. 

Also in the result, please show click through rate as "ctr". (20 points)

In your solution, please provide the SQL query which can best answer the question. Also, please answer 
how many rows are in the returned results of the provided query. Also, please provide the screenshot 
of the first 5 rows in the returned results.

In [ ]:
#Final
data=cur.execute("select Clicked.ad_id, Clicked.platform, Clicked.CNT, cast(Tot.CNT1 as float), (Clicked.CNT/cast(Tot.CNT1 as float)) *100 as ctr from ( select a.ad_id, b.platform, count(*) CNT from clicks a inner join events b on a.display_id = b.display_id where a.clicked = 1 group by ad_id, b.platform ) Clicked, ( select a.ad_id, b.platform, count(*) CNT1 from clicks a inner join events b on a.display_id = b.display_id group by ad_id, b.platform ) Tot where Clicked.ad_id = Tot.ad_id and Clicked.platform = Tot.platform order by ctr desc,Clicked.ad_id, Clicked.platform limit 5 ")
data.fetchall()

[(8386, 3, 1, 1.0, 100.0),
 (8847, 1, 1, 1.0, 100.0),
 (8991, 2, 1, 1.0, 100.0),
 (9231, 1, 1, 1.0, 100.0),
 (9407, 1, 1, 1.0, 100.0)]

In [ ]:
#Final
data=cur.execute("select count(*) from (select Clicked.ad_id, Clicked.platform, Clicked.CNT, cast(Tot.CNT1 as float), (Clicked.CNT/cast(Tot.CNT1 as float)) *100 as ctr from ( select a.ad_id, b.platform, count(*) CNT from clicks a inner join events b on a.display_id = b.display_id where a.clicked = 1 group by ad_id, b.platform ) Clicked, ( select a.ad_id, b.platform, count(*) CNT1 from clicks a inner join events b on a.display_id = b.display_id group by ad_id, b.platform ) Tot where Clicked.ad_id = Tot.ad_id and Clicked.platform = Tot.platform order by ctr desc,Clicked.ad_id, Clicked.platform ) ")
data.fetchall()

[(1406,)]

##Question5 ##

Exam each advertiser's ads performance across different publishers. For each advertiser, we want to see 
how their ads are performed in different publishers. 

Ads performance is measured by click through rate. It 
is defined in the Question 4. publisher_id can be found in documents_meta table. 

Please use 
documents_meta table in your query. In the SQL query result, Please return ad_id, publisher_id, show click 
through rate as "ctr". (20 points)


In your solution, please provide the SQL query *which* can best answer the question. Also, please answer 
how many rows are in the returned results of the provided query. 

In [ ]:
#Final
data=cur.execute("select Clicked.ad_id, Clicked.publisher_id, Clicked.CNT, cast(Tot.CNT1 as float), (Clicked.CNT/cast(Tot.CNT1 as float)) *100 as ctr from ( select a.ad_id, c.publisher_id, count(*) CNT from clicks a inner join events b on a.display_id = b.display_id  inner join documents_meta c on c.document_id=b.document_id where a.clicked = 1 group by ad_id, c.publisher_id ) Clicked, ( select a.ad_id, c.publisher_id, count(*) CNT1 from clicks a inner join events b on a.display_id = b.display_id inner join documents_meta c on c.document_id=b.document_id group by ad_id, c.publisher_id ) Tot where Clicked.ad_id = Tot.ad_id and Clicked.publisher_id = Tot.publisher_id order by ctr desc,Clicked.ad_id, Clicked.publisher_id limit 5")
data.fetchall()

[(3358, 450, 1, 1.0, 100.0),
 (5760, 78, 1, 1.0, 100.0),
 (7432, 579, 1, 1.0, 100.0),
 (7432, 869, 1, 1.0, 100.0),
 (8386, 875, 1, 1.0, 100.0)]

In [ ]:
data=cur.execute("select count(*) from (select Clicked.ad_id, Clicked.publisher_id, Clicked.CNT, cast(Tot.CNT1 as float), (Clicked.CNT/cast(Tot.CNT1 as float)) *100 as ctr from ( select a.ad_id, c.publisher_id, count(*) CNT from clicks a inner join events b on a.display_id = b.display_id  inner join documents_meta c on c.document_id=b.document_id where a.clicked = 1 group by ad_id, c.publisher_id ) Clicked, ( select a.ad_id, c.publisher_id, count(*) CNT1 from clicks a inner join events b on a.display_id = b.display_id inner join documents_meta c on c.document_id=b.document_id group by ad_id, c.publisher_id ) Tot where Clicked.ad_id = Tot.ad_id and Clicked.publisher_id = Tot.publisher_id order by ctr desc,Clicked.ad_id, Clicked.publisher_id )")
data.fetchall()

[(1761,)]

##Question 6##
Retrieve all events displayed in desktop (platform) and in US. 

Please use geo_location field to decide 
whether a display is shown in US or not. 

In the SQL query result, please return all attributes of the display 
event that satisfy the condition. (15 points)


In your solution, please provide the SQL query which can best answer the question. Also, please answer 
how many rows are in the returned results of the provided query. 

In [ ]:
data=cur.execute("select a.* from events a where a.platform=1 and geo_location like 'US>%' limit 5")
data.fetchall()

[(9, '9dddccf70f6067', 1772126, 667, 1, 'US>FL>528'),
 (12, '6fa993bd0e0157', 892701, 718, 1, 'US>TX>612'),
 (13, '7355615832b3af', 1778533, 739, 1, 'US>AZ>753'),
 (16, '30c0ad12b36375', 1727882, 1033, 1, 'US>FL>561'),
 (31, '6a875b33107c5c', 1766961, 2557, 1, 'US>KY>529')]

In [ ]:
data=cur.execute("select count(*) from (select a.* from events a where a.platform=1 and geo_location like 'US>%')")
data.fetchall()

[(490,)]